In [23]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


In [17]:
df = pd.read_csv("IMDB Dataset.csv")  # Ensure the file is in your working directory
print(df.head())
print(df['sentiment'].value_counts())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [18]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)  # remove punctuation/numbers
    return text

df['review'] = df['review'].apply(clean_text)

In [19]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'], df['sentiment'], test_size=0.2, random_state=42
)

In [21]:
vectorizer = TfidfVectorizer(max_features=15000, ngram_range=(1,2), stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()


print("TF-IDF shape:", X_train_tfidf.shape)


TF-IDF shape: (40000, 15000)


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.regularizers import l2  # 👈 Add this line

model = Sequential([
    Dense(1024, kernel_regularizer=l2(0.001), input_shape=(X_train_tfidf.shape[1],)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.5),

    Dense(512, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.4),

    Dense(256, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.3),

    Dense(128, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])


f:\AI_ML Course\Deep Learning\ANN-MLP-implementation\venv\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [27]:
from tensorflow.keras.callbacks import EarlyStopping  

early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=2, 
    restore_best_weights=True
)


In [28]:
history = model.fit(
    X_train_tfidf, y_train,
    epochs=15,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 119s 221ms/step - accuracy: 0.8316 - loss: 2.5394 - val_accuracy: 0.8407 - val_loss: 2.1900
Epoch 2/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 106s 212ms/step - accuracy: 0.9137 - loss: 1.6322 - val_accuracy: 0.8761 - val_loss: 1.4338
Epoch 3/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 105s 211ms/step - accuracy: 0.9263 - loss: 1.1231 - val_accuracy: 0.8761 - val_loss: 1.1581
Epoch 4/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 107s 214ms/step - accuracy: 0.9285 - loss: 0.8925 - val_accuracy: 0.8719 - val_loss: 1.0011
Epoch 5/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 106s 212ms/step - accuracy: 0.9303 - loss: 0.8046 - val_accuracy: 0.8669 - val_loss: 0.9756
Epoch 6/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 203ms/step - accuracy: 0.9348 - loss: 0.7663 - val_accuracy: 0.8714 - val_loss: 0.9680
Epoch 7/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 201ms/step - accuracy: 0.9429 - loss: 0.7470 - val_accuracy: 0.8670 - val_loss: 1.0098
Epoch 8/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 100s 201ms/step - accuracy: 0.9421 -

In [30]:
y_pred = (model.predict(X_test_tfidf) > 0.5).astype("int32")

print("\n✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🔍 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step

✅ Accuracy: 0.8743

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.86      0.87      4961
           1       0.87      0.88      0.88      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000


🔍 Confusion Matrix:
 [[4287  674]
 [ 583 4456]]


In [31]:
# ============================================
# 💾 Save Model and TF-IDF Vectorizer
# ============================================
import joblib

# Save the trained model
model.save("IMDB_Sentiment_Classifier.h5")
print("✅ Model saved successfully as 'IMDB_Sentiment_Classifier.h5'")

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
print("✅ TF-IDF Vectorizer saved successfully as 'tfidf_vectorizer.pkl'")


✅ Model saved successfully as 'IMDB_Sentiment_Classifier.h5'
✅ TF-IDF Vectorizer saved successfully as 'tfidf_vectorizer.pkl'


In [32]:
from tensorflow.keras.models import load_model
import joblib

# Load model and vectorizer
model = load_model("IMDB_Sentiment_Classifier.h5")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Example prediction
sample_text = ["This movie was absolutely fantastic!"]
sample_tfidf = vectorizer.transform(sample_text).toarray()
prediction = (model.predict(sample_tfidf) > 0.5).astype("int32")

print("🧠 Predicted Sentiment:", "Positive" if prediction[0][0] == 1 else "Negative")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
🧠 Predicted Sentiment: Negative
